In [0]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'examples'))
	print(os.getcwd())
except:
	pass


 # TRF Tutorial

 In this notebook we show how to import word onset as a word-level feature and compute TRF from them.

 ## Import EEG

In [0]:
from pyeeg.io import eeglab2mne


 ## Import Word-level features

In [0]:
from pyeeg.io import WordLevelFeatures


 ## Run TRF modelling

In [0]:
from pyeeg.models import TRFEstimator